In [1]:
import random, gc, os, pickle, csv, time

import datasets.utils
import models.utils
from models.cls_oml_ori_v2 import OML
from models.base_models_ori import LabelAwareReplayMemory

import numpy as np

import higher
import torch
import torch.nn.functional as F
from torch.utils import data

# Constants

In [2]:
dataset_order_mapping = {
    1: [2, 0, 3, 1, 4],
    2: [3, 4, 0, 1, 2],
    3: [2, 4, 1, 3, 0],
    4: [0, 2, 1, 4, 3]
}
n_classes = 33
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
# model_path = "/data/model_runs/original_oml/aOML-order1-2022-07-18/OML-order1-id4-2022-07-18_17-53-13.518612.pt"
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr002-2022-07-31/OML-order1-id4-2022-07-31_14-53-46.456804.pt"
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr005-2022-07-31/OML-order1-id4-2022-07-31_18-47-41.477968.pt"
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr005-up20-2022-08-01/OML-order1-id4-2022-08-01_14-45-55.869765.pt"
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2022-07-31/OML-order1-id4-2022-07-31_21-18-36.241546.pt"
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr020-2022-08-16/OML-order1-id4-2022-08-16_11-37-19.424113.pt"
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr050-2022-08-16/OML-order1-id4-2022-08-16_14-16-12.167637.pt"

# v. SR 
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2022-08-29-sr/OML-order1-id4-2022-08-29_18-10-31.695669.pt"
# v. SR Query
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2022-08-30-sr-query/OML-order1-id4-2022-08-30_05-21-18.854228.pt"
model_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2022-14-21-sreversed/OML-order1-id4-2022-12-15_03-55-41.428520.pt"

# memory_path = "/data/model_runs/original_oml/aOML-order1-2022-07-18/OML-order1-id4-2022-07-18_17-53-13.518639_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr002-2022-07-31/OML-order1-id4-2022-07-31_14-53-46.456828_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr005-2022-07-31/OML-order1-id4-2022-07-31_18-47-41.477992_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr005-up20-2022-08-01/OML-order1-id4-2022-08-01_14-45-55.869797_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2022-07-31/OML-order1-id4-2022-07-31_21-18-36.241572_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr020-2022-08-16/OML-order1-id4-2022-08-16_11-37-19.424139_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr050-2022-08-16/OML-order1-id4-2022-08-16_14-16-12.167666_memory.pickle"
# v. SR 
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2022-08-29-sr/OML-order1-id4-2022-08-29_18-10-31.695692_memory.pickle"
# v. SR Query
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2022-08-30-sr-query/OML-order1-id4-2022-08-30_05-21-18.854254_memory.pickle"
memory_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2022-14-21-sreversed/OML-order1-id4-2022-12-15_03-55-41.428551_memory.pickle"


# new_memory_path, ext = os.path.splitext(memory_path)
# new_memory_path = new_memory_path + "_label" + ext

use_db_cache = True
cache_dir = 'tmp'

In [3]:
args = {
    "order": 1,
    "n_epochs": 1,
    "lr": 3e-5,
    "inner_lr": 0.001*10,
    "meta_lr": 3e-5,
    "model": "bert",
    "learner": "oml",
    "mini_batch_size": 16,
    "updates": 5*1,
    "write_prob": 1.0,
    "max_length": 448,
    "seed": 42,
    "replay_rate": 0.01,
    "replay_every": 9600
}
updates = args["updates"]
mini_batch_size = args["mini_batch_size"]
order = args["order"]

In [4]:
torch.manual_seed(args["seed"])
random.seed(args["seed"])
np.random.seed(args["seed"])

# Load Dataset

In [5]:
print('Loading the datasets')
test_datasets = []
for dataset_id in dataset_order_mapping[order]:
    test_dataset_file = os.path.join(cache_dir, f"{dataset_id}.cache")
    if os.path.exists(test_dataset_file):
        with open(test_dataset_file, 'rb') as f:
            test_dataset = pickle.load(f)
    else:
        test_dataset = datasets.utils.get_dataset_test("", dataset_id)
        print('Loaded {}'.format(test_dataset.__class__.__name__))
        test_dataset = datasets.utils.offset_labels(test_dataset)
        pickle.dump(test_dataset, open( test_dataset_file, "wb" ), protocol=pickle.HIGHEST_PROTOCOL)
        print(f"Pickle saved at {test_dataset_file}")
    test_datasets.append(test_dataset)
print('Finished loading all the datasets')

Loading the datasets
Finished loading all the datasets


# Load Model

In [6]:
learner = OML(device=device, n_classes=n_classes, **args)
print('Using {} as learner'.format(learner.__class__.__name__))
learner.load_model(model_path)
with open(memory_path, 'rb') as f:
#     learner.memory = pickle.load(f)
    memory_buffer = pickle.load(f)


2022-12-16 10:44:40,645 - transformers.tokenization_utils_base - INFO - loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /root/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
2022-12-16 10:44:42,055 - transformers.configuration_utils - INFO - loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
2022-12-16 10:44:42,059 - transformers.configuration_utils - INFO - Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer

Using OML as learner


In [7]:
# Setting up task dict for task-aware
memory_buffer.task_dict = {
    0: list(range(5, 9)), # AG
    1: list(range(0, 5)), # Amazon
    2: list(range(0, 5)), # Yelp
    3: list(range(9, 23)), # DBPedia
    4: list(range(23, 33)), # Yahoo
}

In [8]:
# label to task converter - hack, since normally we would just use the task token/identifier
def get_task_from_label(label_idx, task_dict):
    for task_idx, class_list in task_dict.items():
        if label_idx in class_list: return task_idx
    return -1
print(get_task_from_label(8, memory_buffer.task_dict))
# Find mode: https://stackoverflow.com/questions/10797819/finding-the-mode-of-a-list
def mode(array):
    return max(array, key = array.count)
# label list to task id
def get_task_from_label_list(label_list, task_dict):
    return mode([get_task_from_label(label, task_dict) for label in label_list])
print(get_task_from_label_list([1,2,32,1,4,2,0], memory_buffer.task_dict))

0
1


In [9]:
dataclass_mapper = {
    "AGNewsDataset": 0,
    "AmazonDataset": 1,
    "YelpDataset": 2,
    "DBPediaDataset": 3,
    "YahooAnswersDataset": 4
}
dataclass_mapper["AGNewsDataset"]

0

# Testing

Select specific column index per row
https://stackoverflow.com/questions/23435782/numpy-selecting-specific-column-index-per-row-by-using-a-list-of-indexes

In [10]:
def evaluate(dataloader, updates, mini_batch_size, dataname=""):
    learner.rln.eval()
    learner.pln.train()
    
    all_losses, all_predictions, all_labels, all_label_conf = [], [], [], []
    all_adaptation_time = []
    # Get Query set first. and then find supporting support set
    for query_idx, (query_text, query_labels) in enumerate(dataloader):
        print(f"Query ID {query_idx}/{len(dataloader)}")
        # The task id to optimize to for support set
        # task_idx = get_task_from_label_list(query_labels, memory_buffer.task_dict)
        task_idx = dataclass_mapper[dataname]
        
    
        support_set = []
        for _ in range(updates):
            text, labels = memory_buffer.read_batch_task(batch_size=mini_batch_size, task_idx=task_idx, sort_score=True)
            support_set.append((text, labels))

        with higher.innerloop_ctx(learner.pln, learner.inner_optimizer,
                                  copy_initial_weights=False, track_higher_grads=False) as (fpln, diffopt):
            
            INNER_tic = time.time()
            # Inner loop
            task_predictions, task_labels = [], []
            support_loss = []
            for text, labels in support_set:
                labels = torch.tensor(labels).to(device)
                input_dict = learner.rln.encode_text(text)
                _repr = learner.rln(input_dict)
                output = fpln(_repr)
                loss = learner.loss_fn(output, labels)
                diffopt.step(loss)
                pred = models.utils.make_prediction(output.detach())
                support_loss.append(loss.item())
                task_predictions.extend(pred.tolist())
                task_labels.extend(labels.tolist())
            INNER_toc = time.time() - INNER_tic
            all_adaptation_time.append(INNER_toc)

            acc, prec, rec, f1 = models.utils.calculate_metrics(task_predictions, task_labels)

            print('Support set metrics: Loss = {:.4f}, accuracy = {:.4f}, precision = {:.4f}, '
                        'recall = {:.4f}, F1 score = {:.4f}'.format(np.mean(support_loss), acc, prec, rec, f1))

            # Query set is now here!
            query_labels = torch.tensor(query_labels).to(device)
            query_input_dict = learner.rln.encode_text(query_text)
            with torch.no_grad():
                query_repr = learner.rln(query_input_dict)
                query_output = fpln(query_repr) # Output has size of torch.Size([16, 33]) [BATCH, CLASSES]
                query_loss = learner.loss_fn(query_output, query_labels)
            query_loss = query_loss.item()
            # print(output.detach().size())
            # output.detach().max(-1) max on each Batch, which will return [0] max, [1] indices
            query_output_softmax = F.softmax(query_output, -1)
            query_label_conf = query_output_softmax[np.arange(len(query_output_softmax)), query_labels] # Select labels in the softmax of 33 classes

            query_pred = models.utils.make_prediction(query_output.detach())
            query_acc, query_prec, query_rec, query_f1 = models.utils.calculate_metrics(query_pred.tolist(), query_labels.tolist())
            
            print('Query set metrics: Loss = {:.4f}, accuracy = {:.4f}, precision = {:.4f}, '
                'recall = {:.4f}, F1 score = {:.4f}'.format(np.mean(query_loss), query_acc, query_prec, query_rec, query_f1))

            all_losses.append(query_loss)
            all_predictions.extend(query_pred.tolist())
            all_labels.extend(query_labels.tolist())
            all_label_conf.extend(query_label_conf.tolist())

    acc, prec, rec, f1 = models.utils.calculate_metrics(all_predictions, all_labels)
    print('Test metrics: Loss = {:.4f}, accuracy = {:.4f}, precision = {:.4f}, recall = {:.4f}, '
                'F1 score = {:.4f}'.format(np.mean(all_losses), acc, prec, rec, f1))
    return acc, prec, rec, f1, all_predictions, all_labels, all_label_conf, all_adaptation_time

In [11]:
tic = time.time()
print('----------Testing on test set starts here----------')

accuracies, precisions, recalls, f1s = [], [], [], []
all_adapt_time = []
# Data for Visualization: [data_idx, label, label_conf, pred]
data_for_visual = []

for test_dataset in test_datasets:
    print('Testing on {}'.format(test_dataset.__class__.__name__))
    test_dataloader = data.DataLoader(test_dataset, batch_size=mini_batch_size, shuffle=False,
                                      collate_fn=datasets.utils.batch_encode)
    acc, prec, rec, f1, all_pred, all_label, all_label_conf, all_adaptation_time = evaluate(dataloader=test_dataloader, updates=updates, 
                                                mini_batch_size=mini_batch_size, dataname=test_dataset.__class__.__name__)
    
    data_ids = [test_dataset.__class__.__name__ + str(i) for i in range(len(all_label))]
    data_for_visual.extend(list(zip(data_ids, all_label, all_label_conf, all_pred)))
    all_adapt_time.extend(all_adaptation_time)
#     print(data_ids)
#     print(all_label)
#     raise Exception("BREAKPOINT")
    
    accuracies.append(acc)
    precisions.append(prec)
    recalls.append(rec)
    f1s.append(f1)


print()
print("COPY PASTA - not really but ok")
for row in accuracies:
    print(row)
print()
print('Overall test metrics: Accuracy = {:.4f}, precision = {:.4f}, recall = {:.4f}, '
            'F1 score = {:.4f}'.format(np.mean(accuracies), np.mean(precisions), np.mean(recalls), np.mean(f1s)))

toc = time.time() - tic
print(f"Total Time used: {toc//60} minutes")

----------Testing on test set starts here----------
Testing on YelpDataset
Query ID 0/475
Support set metrics: Loss = 4.6367, accuracy = 0.0250, precision = 0.2000, recall = 0.0233, F1 score = 0.0417
Query set metrics: Loss = 1.6799, accuracy = 0.5625, precision = 0.9200, recall = 0.5867, F1 score = 0.6800
Query ID 1/475
Support set metrics: Loss = 4.4972, accuracy = 0.0875, precision = 0.6500, recall = 0.0833, F1 score = 0.1456
Query set metrics: Loss = 1.3691, accuracy = 0.6250, precision = 0.5667, recall = 0.6000, F1 score = 0.5485
Query ID 2/475
Support set metrics: Loss = 4.7581, accuracy = 0.0375, precision = 0.2333, recall = 0.0367, F1 score = 0.0626
Query set metrics: Loss = 1.8673, accuracy = 0.2500, precision = 0.2900, recall = 0.2167, F1 score = 0.2333
Query ID 3/475
Support set metrics: Loss = 4.5033, accuracy = 0.0125, precision = 0.1000, recall = 0.0100, F1 score = 0.0182
Query set metrics: Loss = 1.6916, accuracy = 0.5625, precision = 0.7333, recall = 0.5333, F1 score = 

Support set metrics: Loss = 4.4991, accuracy = 0.0125, precision = 0.1000, recall = 0.0133, F1 score = 0.0235
Query set metrics: Loss = 1.7859, accuracy = 0.4375, precision = 0.7917, recall = 0.4417, F1 score = 0.5030
Query ID 36/475
Support set metrics: Loss = 4.4430, accuracy = 0.0750, precision = 0.5333, recall = 0.0733, F1 score = 0.1264
Query set metrics: Loss = 1.8380, accuracy = 0.4375, precision = 0.4933, recall = 0.3000, F1 score = 0.3597
Query ID 37/475
Support set metrics: Loss = 4.3442, accuracy = 0.0750, precision = 0.8000, recall = 0.0767, F1 score = 0.1353
Query set metrics: Loss = 2.4623, accuracy = 0.2500, precision = 0.5667, recall = 0.3567, F1 score = 0.3371
Query ID 38/475
Support set metrics: Loss = 4.4572, accuracy = 0.0625, precision = 0.6333, recall = 0.0600, F1 score = 0.1083
Query set metrics: Loss = 1.5969, accuracy = 0.5000, precision = 0.5400, recall = 0.5067, F1 score = 0.4387
Query ID 39/475
Support set metrics: Loss = 4.3095, accuracy = 0.0875, precision

Query set metrics: Loss = 2.1820, accuracy = 0.0000, precision = 0.0000, recall = 0.0000, F1 score = 0.0000
Query ID 71/475
Support set metrics: Loss = 4.5533, accuracy = 0.0750, precision = 0.5667, recall = 0.0700, F1 score = 0.1184
Query set metrics: Loss = 1.8556, accuracy = 0.3750, precision = 0.3533, recall = 0.3500, F1 score = 0.3310
Query ID 72/475
Support set metrics: Loss = 4.5584, accuracy = 0.0375, precision = 0.3000, recall = 0.0333, F1 score = 0.0583
Query set metrics: Loss = 1.9728, accuracy = 0.3750, precision = 0.6000, recall = 0.4100, F1 score = 0.3600
Query ID 73/475
Support set metrics: Loss = 4.4975, accuracy = 0.0750, precision = 0.7333, recall = 0.0733, F1 score = 0.1318
Query set metrics: Loss = 2.0547, accuracy = 0.1875, precision = 0.2571, recall = 0.1733, F1 score = 0.1467
Query ID 74/475
Support set metrics: Loss = 4.5464, accuracy = 0.0750, precision = 0.4333, recall = 0.0767, F1 score = 0.1293
Query set metrics: Loss = 2.1743, accuracy = 0.3125, precision =

Query set metrics: Loss = 1.5338, accuracy = 0.5625, precision = 0.5333, recall = 0.6000, F1 score = 0.5000
Query ID 106/475
Support set metrics: Loss = 4.4615, accuracy = 0.0375, precision = 0.4667, recall = 0.0400, F1 score = 0.0722
Query set metrics: Loss = 1.9225, accuracy = 0.3125, precision = 0.4238, recall = 0.4333, F1 score = 0.3371
Query ID 107/475
Support set metrics: Loss = 4.4965, accuracy = 0.0625, precision = 0.5667, recall = 0.0633, F1 score = 0.1095
Query set metrics: Loss = 1.9400, accuracy = 0.2500, precision = 0.3333, recall = 0.2333, F1 score = 0.2467
Query ID 108/475
Support set metrics: Loss = 4.5083, accuracy = 0.0375, precision = 0.4333, recall = 0.0367, F1 score = 0.0631
Query set metrics: Loss = 2.2348, accuracy = 0.0625, precision = 0.1000, recall = 0.0667, F1 score = 0.0800
Query ID 109/475
Support set metrics: Loss = 4.5154, accuracy = 0.0125, precision = 0.1000, recall = 0.0100, F1 score = 0.0182
Query set metrics: Loss = 1.6801, accuracy = 0.3125, precisi

Query set metrics: Loss = 1.9273, accuracy = 0.2500, precision = 0.2600, recall = 0.3000, F1 score = 0.2256
Query ID 141/475
Support set metrics: Loss = 4.2409, accuracy = 0.1000, precision = 0.7667, recall = 0.1000, F1 score = 0.1747
Query set metrics: Loss = 1.8425, accuracy = 0.4375, precision = 0.5000, recall = 0.3500, F1 score = 0.3714
Query ID 142/475
Support set metrics: Loss = 4.3340, accuracy = 0.0500, precision = 0.5500, recall = 0.0500, F1 score = 0.0892
Query set metrics: Loss = 1.7330, accuracy = 0.3750, precision = 0.6800, recall = 0.4300, F1 score = 0.4832
Query ID 143/475
Support set metrics: Loss = 4.4898, accuracy = 0.0625, precision = 0.4800, recall = 0.0600, F1 score = 0.1041
Query set metrics: Loss = 1.7859, accuracy = 0.3750, precision = 0.4000, recall = 0.3033, F1 score = 0.3405
Query ID 144/475
Support set metrics: Loss = 4.3898, accuracy = 0.0375, precision = 0.5000, recall = 0.0400, F1 score = 0.0735
Query set metrics: Loss = 2.3191, accuracy = 0.0625, precisi

...

Support set metrics: Loss = 0.5620, accuracy = 0.7875, precision = 0.8166, recall = 0.7400, F1 score = 0.7528
Query set metrics: Loss = 1.0172, accuracy = 0.6875, precision = 0.7143, recall = 0.6810, F1 score = 0.6738
Query ID 309/475
Support set metrics: Loss = 0.5389, accuracy = 0.8500, precision = 0.8937, recall = 0.8600, F1 score = 0.8515
Query set metrics: Loss = 0.7750, accuracy = 0.8125, precision = 0.9074, recall = 0.8704, F1 score = 0.8481
Query ID 310/475
Support set metrics: Loss = 0.4933, accuracy = 0.9000, precision = 0.9110, recall = 0.8900, F1 score = 0.8856
Query set metrics: Loss = 0.4250, accuracy = 0.8125, precision = 0.9286, recall = 0.8524, F1 score = 0.8603
Query ID 311/475
Support set metrics: Loss = 0.5818, accuracy = 0.8375, precision = 0.8568, recall = 0.8200, F1 score = 0.8254
Query set metrics: Loss = 0.8344, accuracy = 0.8125, precision = 0.8750, recall = 0.8542, F1 score = 0.8083
Query ID 312/475
Support set metrics: Loss = 0.6033, accuracy = 0.8000, preci

Support set metrics: Loss = 0.5454, accuracy = 0.8500, precision = 0.8869, recall = 0.8500, F1 score = 0.8497
Query set metrics: Loss = 0.6241, accuracy = 0.8750, precision = 0.9688, recall = 0.9062, F1 score = 0.9271
Query ID 344/475
Support set metrics: Loss = 0.5282, accuracy = 0.8375, precision = 0.8837, recall = 0.8200, F1 score = 0.8290
Query set metrics: Loss = 1.2134, accuracy = 0.5625, precision = 0.4167, recall = 0.5000, F1 score = 0.4167
Query ID 345/475
Support set metrics: Loss = 0.5615, accuracy = 0.8000, precision = 0.8459, recall = 0.8000, F1 score = 0.8073
Query set metrics: Loss = 1.0519, accuracy = 0.6250, precision = 0.6875, recall = 0.7292, F1 score = 0.6417
Query ID 346/475
Support set metrics: Loss = 0.5670, accuracy = 0.8250, precision = 0.8481, recall = 0.8100, F1 score = 0.8070
Query set metrics: Loss = 0.8483, accuracy = 0.8125, precision = 0.7407, recall = 0.8519, F1 score = 0.7704
Query ID 347/475
Support set metrics: Loss = 0.5632, accuracy = 0.8500, preci

Support set metrics: Loss = 0.5578, accuracy = 0.8500, precision = 0.8726, recall = 0.8300, F1 score = 0.8355
Query set metrics: Loss = 0.4463, accuracy = 0.8750, precision = 0.7963, recall = 0.8519, F1 score = 0.8148
Query ID 379/475
Support set metrics: Loss = 0.5404, accuracy = 0.8875, precision = 0.8879, recall = 0.8800, F1 score = 0.8732
Query set metrics: Loss = 0.7759, accuracy = 0.8125, precision = 0.7500, recall = 0.7188, F1 score = 0.7321
Query ID 380/475
Support set metrics: Loss = 0.5487, accuracy = 0.8375, precision = 0.8583, recall = 0.8200, F1 score = 0.8171
Query set metrics: Loss = 0.9119, accuracy = 0.6875, precision = 0.5972, recall = 0.5417, F1 score = 0.5543
Query ID 381/475
Support set metrics: Loss = 0.5906, accuracy = 0.8250, precision = 0.8600, recall = 0.7900, F1 score = 0.7967
Query set metrics: Loss = 0.5549, accuracy = 0.8750, precision = 0.8125, recall = 0.8542, F1 score = 0.8220
Query ID 382/475
Support set metrics: Loss = 0.4880, accuracy = 0.9000, preci

Support set metrics: Loss = 0.5462, accuracy = 0.8000, precision = 0.8413, recall = 0.8000, F1 score = 0.7976
Query set metrics: Loss = 0.6101, accuracy = 0.7500, precision = 0.7222, recall = 0.7778, F1 score = 0.7222
Query ID 414/475
Support set metrics: Loss = 0.5813, accuracy = 0.7875, precision = 0.7771, recall = 0.7500, F1 score = 0.7518
Query set metrics: Loss = 0.2707, accuracy = 0.9375, precision = 0.9583, recall = 0.9375, F1 score = 0.9333
Query ID 415/475
Support set metrics: Loss = 0.5440, accuracy = 0.8500, precision = 0.8733, recall = 0.8600, F1 score = 0.8443
Query set metrics: Loss = 0.1620, accuracy = 0.9375, precision = 0.9444, recall = 0.9444, F1 score = 0.9259
Query ID 416/475
Support set metrics: Loss = 0.5571, accuracy = 0.8375, precision = 0.8666, recall = 0.8100, F1 score = 0.8030
Query set metrics: Loss = 0.8509, accuracy = 0.7500, precision = 0.8571, recall = 0.8571, F1 score = 0.8095
Query ID 417/475
Support set metrics: Loss = 0.5416, accuracy = 0.8375, preci

Support set metrics: Loss = 0.5172, accuracy = 0.8625, precision = 0.8955, recall = 0.8500, F1 score = 0.8559
Query set metrics: Loss = 0.8254, accuracy = 0.6250, precision = 0.6406, recall = 0.5417, F1 score = 0.5628
Query ID 449/475
Support set metrics: Loss = 0.5761, accuracy = 0.8000, precision = 0.8400, recall = 0.7700, F1 score = 0.7597
Query set metrics: Loss = 0.8473, accuracy = 0.7500, precision = 0.8542, recall = 0.8125, F1 score = 0.7792
Query ID 450/475
Support set metrics: Loss = 0.5130, accuracy = 0.9250, precision = 0.9394, recall = 0.9200, F1 score = 0.9242
Query set metrics: Loss = 0.8773, accuracy = 0.7500, precision = 0.7667, recall = 0.8000, F1 score = 0.7571
Query ID 451/475
Support set metrics: Loss = 0.5577, accuracy = 0.8500, precision = 0.8885, recall = 0.8400, F1 score = 0.8424
Query set metrics: Loss = 0.3851, accuracy = 0.8750, precision = 0.7857, recall = 0.8214, F1 score = 0.7891
Query ID 452/475
Support set metrics: Loss = 0.6134, accuracy = 0.7875, preci

In [12]:
_model_path0 = os.path.splitext(model_path)[0]
csv_filename = _model_path0 + "_update"+ str(updates) +"_results.csv" # for selective replay
with open(csv_filename, 'w') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(["data_idx", "label", "label_conf", "pred"])
    csv_writer.writerows(data_for_visual)
print(f"Done writing CSV File at {csv_filename}")

Done writing CSV File at /data/model_runs/original_oml/aOML-order1-inlr010-2022-14-21-sreversed/OML-order1-id4-2022-12-15_03-55-41.428520_update5_results.csv


In [13]:
# Log Time for Inference
_model_path0 = os.path.splitext(model_path)[0]
time_txt_filename = _model_path0 + "_update"+ str(updates) +"_time_inference.csv" 
with open(time_txt_filename, 'w') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(["time_id", "time"])
    csv_writer.writerow(["Total Time", f"{toc//60} minutes"])
    csv_writer.writerow(["mean Adapt Time", f"{np.mean(all_adapt_time)} s"])
print(f"Done writing Time CSV File at {time_txt_filename}")

Done writing Time CSV File at /data/model_runs/original_oml/aOML-order1-inlr010-2022-14-21-sreversed/OML-order1-id4-2022-12-15_03-55-41.428520_update5_time_inference.csv
